In [2]:
# セル1: ライブラリのインポート
import pandas as pd
import logging
from datetime import datetime
import os
import yaml 
from pathlib import Path

In [3]:
# セル2: 統合用クラスの定義
class DataIntegrator:
    """SP-APIとKeepa APIのデータを統合するクラス"""
    
    def __init__(self, config_path=None):
        """
        初期化と設定読み込み
        
        Parameters:
        -----------
        config_path : str, optional
            設定ファイルのパス。指定がない場合はデフォルトパスを使用
        """
        # プロジェクトルートディレクトリの検出
        self.root_dir = self._find_project_root()
        
        # データディレクトリとログディレクトリのパスを設定
        self.data_dir = os.path.join(self.root_dir, 'data')
        self.log_dir = os.path.join(self.root_dir, 'logs')
        
        # ディレクトリの存在確認
        if not os.path.exists(self.data_dir):
            os.makedirs(self.data_dir)
        if not os.path.exists(self.log_dir):
            os.makedirs(self.log_dir)
            
        # 設定ファイルの読み込み
        self.config = self._load_config(config_path)
        
        # ログ機能の設定
        self.setup_logging()

    def _find_project_root(self):
        """プロジェクトのルートディレクトリを検出する"""
        # 現在のファイルの絶対パスを取得
        current_dir = os.path.abspath(os.getcwd())
        
        # 親ディレクトリを探索
        path = Path(current_dir)
        while True:
            # .gitディレクトリがあればそれをルートとみなす
            if (path / '.git').exists():
                return str(path)
            
            # プロジェクトのルートを示す他のファイル/ディレクトリの存在チェック
            if (path / 'setup.py').exists() or (path / 'README.md').exists():
                return str(path)
            
            # これ以上上の階層がない場合は現在のディレクトリを返す
            if path.parent == path:
                return str(path)
            
            # 親ディレクトリへ
            path = path.parent
    
    def _load_config(self, config_path=None):
        """設定ファイルを読み込む"""
        if config_path is None:
            config_path = os.path.join(self.root_dir, 'config', 'settings.yaml')
            
        try:
            with open(config_path, 'r', encoding='utf-8') as f:
                config = yaml.safe_load(f)
                
            # データ統合設定の存在確認
            if 'data_integration' not in config:
                config['data_integration'] = {}
                
            # output設定の初期化（なければデフォルト値を設定）
            if 'output' not in config['data_integration']:
                config['data_integration']['output'] = {
                    'sp_api_input': 'sp_api_output_filtered.csv',
                    'keepa_input': 'keepa_output.csv',
                    'output_file': 'integrated_data.csv'
                }
                
            # sources設定の初期化（なければ空リストを設定）
            if 'sources' not in config['data_integration']:
                config['data_integration']['sources'] = []
                
            print(f"設定ファイルを読み込みました: {config_path}")
            return config
            
        except Exception as e:
            print(f"設定ファイルの読み込みに失敗: {str(e)}")
            # 読み込み失敗時はデフォルト設定を返す
            return {
                'data_integration': {
                    'output': {
                        'sp_api_input': 'sp_api_output_filtered.csv',
                        'keepa_input': 'keepa_output.csv',
                        'output_file': 'integrated_data.csv'
                    },
                    'sources': []
                }
            }
    
    
    
    def setup_logging(self):
        """ログ機能のセットアップ"""
        log_file = os.path.join(self.log_dir, f'integration_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')
        logging.basicConfig(
            filename=log_file,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            encoding='utf-8'
        )
        logging.info(f"ログファイル: {log_file}")
        logging.info("データ統合処理を開始します")
        print(f"📄 ログファイル: {log_file}")

    def load_data(self, sp_api_file=None, keepa_file=None):
        """
        CSVファイルの読み込み
        
        Parameters:
        -----------
        sp_api_file : str, optional
            SP-APIデータのファイルパス（デフォルトは設定ファイルから）
        keepa_file : str, optional
            Keepaデータのファイルパス（デフォルトは設定ファイルから）
            
        Returns:
        --------
        tuple
            (sp_df, keepa_df) - 読み込んだデータフレーム
        """
        try:
            # 入力ファイル名が指定されていない場合は設定ファイルから取得
            if sp_api_file is None:
                sp_api_file = self.config['data_integration']['output']['sp_api_input']
                
            if keepa_file is None:
                keepa_file = self.config['data_integration']['output']['keepa_input']
            
            # 相対パスの場合はdataディレクトリを基準にする
            if not os.path.isabs(sp_api_file):
                sp_api_file = os.path.join(self.data_dir, sp_api_file)
                
            if not os.path.isabs(keepa_file):
                keepa_file = os.path.join(self.data_dir, keepa_file)
            
            # ファイルの存在確認とエラーメッセージの改善
            if not os.path.exists(sp_api_file):
                error_msg = f"SP-APIファイルが見つかりません: {sp_api_file}"
                print(f"⚠️ {error_msg}")
                raise FileNotFoundError(error_msg)
                
            if not os.path.exists(keepa_file):
                error_msg = f"Keepaファイルが見つかりません: {keepa_file}"
                print(f"⚠️ {error_msg}")
                raise FileNotFoundError(error_msg)
            
            # SP-APIデータの読み込み
            sp_df = pd.read_csv(sp_api_file, encoding='utf-8-sig')
            logging.info(f"SP-APIデータを読み込みました: {len(sp_df)}件")
            
            # Keepaデータの読み込み
            keepa_df = pd.read_csv(keepa_file, encoding='utf-8-sig')
            logging.info(f"Keepaデータを読み込みました: {len(keepa_df)}件")
            
            return sp_df, keepa_df
            
        except Exception as e:
            logging.error(f"データ読み込みエラー: {str(e)}")
            raise


    # 新しいメソッドを追加
    def load_source_data(self, source_config):
        """
        ソース設定に基づいてCSVファイルを読み込む
        
        Parameters:
        -----------
        source_config : dict
            ソース設定情報
            
        Returns:
        --------
        dict
            ファイル名をキー、データフレームを値とする辞書
        """
        result = {}
        
        try:
            files = source_config.get('files', [])
            key_column = source_config.get('key_column', 'JAN')
            
            # JAN列の代替名リスト（よくある命名パターン）
            jan_column_alternatives = ['JAN', 'JANコード', 'jan', 'jancode', 'jan_code', 'ean', 'EAN']
            
            for file in files:
                try:
                    # 相対パスの場合はプロジェクトのdataディレクトリを基準にする
                    file_path = file
                    if not os.path.isabs(file_path):
                        file_path = os.path.join(self.data_dir, file_path)
                    
                    # ファイルの存在確認
                    if not os.path.exists(file_path):
                        logging.warning(f"ファイルが見つかりません: {file_path}。スキップします。")
                        print(f"⚠️ ファイルが見つかりません: {file_path}。スキップします。")
                        continue
                    
                    # CSVファイルを読み込む
                    df = pd.read_csv(file_path, encoding='utf-8-sig')
                    
                    # 実際のキー列を特定
                    actual_key_column = key_column  # デフォルト値
                    
                    # 設定されたキー列がない場合は代替名から探す
                    if key_column not in df.columns:
                        # 代替名リストから列名を探す
                        for alt_column in jan_column_alternatives:
                            if alt_column in df.columns:
                                actual_key_column = alt_column
                                print(f"ℹ️ {file}では「{key_column}」の代わりに「{actual_key_column}」を使用します")
                                break
                    
                    # キー列の存在確認
                    if actual_key_column not in df.columns:
                        logging.warning(f"キー列 '{key_column}' またはその代替名が {file} に見つかりません。スキップします。")
                        print(f"⚠️ キー列 '{key_column}' またはその代替名が {file} に見つかりません。スキップします。")
                        print(f"  利用可能な列: {', '.join(df.columns)}")
                        continue
                    
                    # 結果に追加
                    result[file] = {
                        'df': df,
                        'key_column': actual_key_column
                    }
                    
                    logging.info(f"ソースデータを読み込みました: {file} ({len(df)}件), キー列: {actual_key_column}")
                    print(f"📊 {file}を読み込みました: {len(df)}件, キー列: {actual_key_column}")
                    
                except Exception as e:
                    logging.error(f"ファイル {file} の読み込みエラー: {str(e)}")
                    print(f"⚠️ {file}の読み込みエラー: {str(e)}")
                    continue
                    
            return result
            
        except Exception as e:
            logging.error(f"ソースデータ読み込み全体エラー: {str(e)}")
            return {}


    def merge_source_data(self, base_df, source_data, source_config):
        """
        ベースのデータフレームにソースデータを結合する
        
        Parameters:
        -----------
        base_df : pandas.DataFrame
            ベースのデータフレーム
        source_data : dict
            ファイル名をキー、データフレーム情報を値とする辞書
        source_config : dict
            ソース設定情報
            
        Returns:
        --------
        pandas.DataFrame
            結合後のデータフレーム
        """
        result_df = base_df.copy()
        source_type = source_config.get('type', '不明')
        
        # プレフィックスをサイト名に基づいて設定
        if source_type.lower() == 'netsea':
            prefix = 'ネッシー_'
        elif source_type.lower() == 'sudeli':
            prefix = 'スーデリ_'
        else:
            # その他のソースタイプの場合はそのまま使用
            prefix = source_config.get('prefix', '')
        
        try:
            # ベースDFにJAN列が存在するか確認
            if 'JAN' not in result_df.columns:
                logging.warning("ベースデータにJAN列がありません。結合できない可能性があります。")
                print("⚠️ ベースデータにJAN列がありません。結合できない可能性があります。")
                
                # JAN列がない場合、元コード列からJANを取得してみる
                if '元コード' in result_df.columns:
                    # コードタイプがEANであれば元コードをJANとして使用
                    if 'コードタイプ' in result_df.columns:
                        result_df['JAN'] = result_df.apply(
                            lambda row: row['元コード'] if row['コードタイプ'] == 'EAN' else None, 
                            axis=1
                        )
                        print("📊 元コードとコードタイプからJAN列を作成しました")
            
            # 統合するソースファイルごとに個別に処理
            for file, data in source_data.items():
                source_df = data['df']
                key_column = data['key_column']
                
                # キー列の値を文字列に変換（数値の場合があるため）
                source_df[key_column] = source_df[key_column].astype(str)
                if 'JAN' in result_df.columns:
                    result_df['JAN'] = result_df['JAN'].astype(str)
                
                # JANコードをキーに結合
                logging.info(f"'{key_column}'列を'JAN'として結合: {file}")
                print(f"📊 {source_type}データから'{key_column}'列を'JAN'として結合: {file}")
                
                # 結合前にマッチするJANコードの数を確認
                if 'JAN' in result_df.columns:
                    # ベースデータのJANリスト
                    base_jans = set(result_df['JAN'].dropna().unique())
                    # ソースデータのJANリスト
                    source_jans = set(source_df[key_column].dropna().unique())
                    # 共通するJANの数
                    common_jans = base_jans.intersection(source_jans)
                    
                    # 重複するJANをチェック
                    duplicate_jans = source_df[source_df[key_column].duplicated(keep=False)][key_column].unique()
                    if len(duplicate_jans) > 0:
                        duplicate_count = len(duplicate_jans)
                        example_duplicates = list(duplicate_jans)[:3]  # 最大3つまで表示
                        print(f"ℹ️ {file}内に{duplicate_count}件の重複JANを検出: {', '.join(example_duplicates)}など")
                        print(f"ℹ️ 重複JANは各JANの最初のデータのみを使用します")
                    
                    # 重要な修正：各JANの最初のエントリのみを保持する
                    # drop_duplicates()メソッドのkeep='first'引数で最初の行のみを残す
                    source_df_unique = source_df.drop_duplicates(subset=[key_column], keep='first')
                    
                    # 重複削除後の結果を表示
                    removed_count = len(source_df) - len(source_df_unique)
                    if removed_count > 0:
                        print(f"ℹ️ 重複を除去: {len(source_df)}行 → {len(source_df_unique)}行 ({removed_count}行削除)")
                    
                    # マッチするJANの例を表示（最大5つ）
                    if common_jans:
                        example_jans = list(common_jans)[:5]
                        print(f"ℹ️ マッチするJANの例: {', '.join(example_jans)}")
                        
                        # マッチするJANがある場合のみ処理を続行
                        # 列名にプレフィックスを追加
                        source_df_renamed = source_df_unique.copy()  # 重複除去済みのデータフレームを使用
                        rename_dict = {}
                        
                        # キー列以外の列名にプレフィックスを追加
                        for col in source_df_unique.columns:
                            if col != key_column:
                                rename_dict[col] = f"{prefix}{col}"
                        
                        source_df_renamed = source_df_renamed.rename(columns=rename_dict)
                        
                        # 結合前の列数とデータ数を記録
                        pre_merge_columns = len(result_df.columns)
                        
                        # マッチしたJANコードを持つ行のみにフィルタリング
                        filtered_source_df = source_df_renamed[source_df_renamed[key_column].isin(common_jans)]
                        
                        if not filtered_source_df.empty:
                            # 結合実行
                            result_df = pd.merge(
                                result_df,
                                filtered_source_df,
                                left_on='JAN',
                                right_on=key_column,
                                how='left',
                                suffixes=('', f'_{file}')  # 重複列の処理
                            )
                            
                            # 結合結果のチェック
                            post_merge_columns = len(result_df.columns)
                            added_columns = post_merge_columns - pre_merge_columns
                            
                            # 重複キー列を削除
                            if key_column != 'JAN' and key_column in result_df.columns:
                                result_df = result_df.drop(columns=[key_column])
                            
                            # 実際にマッチしたデータの確認
                            match_count = 0
                            if added_columns > 0:
                                # 追加された最初の列を見つける
                                for col in result_df.columns[-added_columns:]:
                                    if col in result_df.columns:
                                        match_count = result_df[col].notna().sum()
                                        break
                            
                            print(f"✅ 結合完了: {len(common_jans)}件のJANがマッチ、{match_count}行のデータに情報追加、{added_columns}列追加")
                            logging.info(f"{file}のデータを結合しました: マッチJAN {len(common_jans)}件、マッチ行 {match_count}件、列数 {added_columns}列追加")
                        else:
                            print(f"⚠️ マッチするJANコードがフィルタリング後に残りませんでした。結合をスキップします。")
                    else:
                        print(f"⚠️ マッチするJANコードがありません。結合をスキップします。")
                else:
                    print(f"⚠️ 結合をスキップ: ベースデータにJAN列がありません")
                    logging.warning(f"結合をスキップ: ベースデータにJAN列がありません")
                
            print(f"✅ {source_type}データの結合完了: 現在 {len(result_df.columns)}列")
            return result_df
            
        except Exception as e:
            logging.error(f"ソースデータ結合エラー: {str(e)}")
            import traceback
            traceback.print_exc()
            return base_df




    

    def merge_data(self, sp_df, keepa_df):
        """データの結合"""
        try:
            # ASINをキーにして結合
            merged_df = pd.merge(
                sp_df,
                keepa_df,
                on='ASIN',
                how='outer',
                suffixes=('_sp', '_keepa')
            )
            
            logging.info(f"データを結合しました: {len(merged_df)}件")
            return merged_df
            
        except Exception as e:
            logging.error(f"データ結合エラー: {str(e)}")
            raise

    def rearrange_columns(self, df):
        """
        カラムの並び替え
        
        指定された列順序のカラムを先頭に配置し、それ以外のカラムは末尾に保持します
        
        Parameters:
        -----------
        df : pandas.DataFrame
            並び替え対象のデータフレーム
            
        Returns:
        --------
        pandas.DataFrame
            列が並び替えられたデータフレーム
        """
        try:
            # 望ましい列順を定義
            column_order = [
                # 基本情報1
                'ASIN', 'JAN', '商品名', 'カテゴリー', 'メーカー型番', 'レビュー有無', 
                'メーカー名', 'ブランド名', '総出品者数', 'セット数', '商品追跡日', 
                '商品発売日', '追跡開始からの経過日数', 'アダルト商品対象',
    
                # 基本情報2
                '参考価格', 'パッケージ最長辺', 'パッケージ中辺', 'パッケージ最短辺', 
                'パッケージ重量', '現在ランキング', '30日間平均ランキング', 
                '90日間平均ランキング', '180日間平均ランキング', 'amazonURL', 
                'KeepaURL', 'バリエーションASIN',
    
                # 価格情報
                'Amazon価格', 'カート価格', 'カート価格送料', 'カート価格のポイント', 
                'リードタイム（時間）', 'FBA最安値', 'FBA最安値のポイント', 
                '自己発送最安値', '自己発送最安値の送料', '自己発送最安値のポイント', 
                'FBA_販売手数料', 'FBA_配送代行手数料',
    
                # 出品者情報
                'amazon本体有無1', 'amazon本体有無2', 'FBA数', '自己発送数', 
                'FBA最安値出品者数', '自己発送最安値出品者数', 
                'amazon_30日間在庫切れ率', 'amazon_90日間在庫切れ率',
    
                # 販売数情報
                '30日間_総販売数', '30日間_新品販売数', '30日間_中古販売数', 
                '30日間_コレクター販売数', 'Keepa30日間販売数', 
                '90日間_総販売数', '90日間_新品販売数', '90日間_中古販売数', 
                '90日間_コレクター販売数', 'Keepa90日間販売数',
                '180日間_総販売数', '180日間_新品販売数', '180日間_中古販売数', 
                '180日間_コレクター販売数', 'Keepa180日間販売数',
    
                # 価格履歴
                'amazon価格_現在価格', 'amazon価格_最高価格', 'amazon価格_最低価格',
                'amazon価格_30日平均価格', 'amazon価格_90日平均価格', 
                'amazon価格_180日平均価格', '新品価格_現在価格', '新品価格_最高価格',
                '新品価格_最低価格', '新品価格_30日平均価格', '新品価格_90日平均価格',
                '新品価格_180日平均価格',
    
                # その他
                '画像URL', '元コード', 'コードタイプ'
            ]
            
            # 存在する列のみを抽出（エラー防止のため）
            specified_columns = [col for col in column_order if col in df.columns]
            
            # 指定されていない残りの列（追加された列など）を取得
            remaining_columns = [col for col in df.columns if col not in column_order]
            
            # 指定列 + 残りの列の順で新しい列順を作成
            new_column_order = specified_columns + remaining_columns
            
            # 並び替えを実行
            df = df[new_column_order]
            
            # 結果をログに記録
            logging.info(f"カラムを並び替えました: 指定列 {len(specified_columns)}列 + 追加列 {len(remaining_columns)}列")
            print(f"📊 カラム並び替え: 指定列 {len(specified_columns)}列 + 追加列 {len(remaining_columns)}列")
            
            return df
            
        except Exception as e:
            logging.error(f"カラム並び替えエラー: {str(e)}")
            raise

    def save_data(self, df, output_file=None):
        """
        統合データの保存
        
        Parameters:
        -----------
        df : pandas.DataFrame
            保存するデータフレーム
        output_file : str, optional
            出力ファイル名（デフォルトは設定ファイルから）
        """
        try:
            # 出力ファイル名が指定されていない場合は設定ファイルから取得
            if output_file is None:
                output_file = self.config['data_integration']['output']['output_file']
                
            # 相対パスの場合はデータディレクトリを基準にする
            if not os.path.isabs(output_file):
                output_file = os.path.join(self.data_dir, output_file)
                
            # 出力ディレクトリの存在確認
            output_dir = os.path.dirname(output_file)
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
                
            df.to_csv(output_file, index=False, encoding='utf-8-sig')
            logging.info(f"統合データを保存しました: {output_file}")
            print(f"✅ {len(df)}件のデータを {output_file} に保存しました")
            
        except Exception as e:
            logging.error(f"データ保存エラー: {str(e)}")
            raise



In [4]:
# # 現在のカラム確認用コード
# print("=== SP-APIのカラム ===")
# sp_df = pd.read_csv('sp_api_output_filtered.csv', encoding='utf-8-sig')
# print(sp_df.columns.tolist())

# print("\n=== Keepaのカラム ===")
# keepa_df = pd.read_csv('keepa_output.csv', encoding='utf-8-sig')
# print(keepa_df.columns.tolist())

In [5]:
# セル3: 実行用コード
if __name__ == "__main__":
    try:
        # インテグレーターのインスタンス作成
        integrator = DataIntegrator()
        
        # 各種パスの確認と表示
        print(f"📂 プロジェクトルートディレクトリ: {integrator.root_dir}")
        print(f"📂 データディレクトリ: {integrator.data_dir}")
        print(f"📂 ログディレクトリ: {integrator.log_dir}")
        
        # 設定ファイルから読み込んだファイル名を表示
        config = integrator.config['data_integration']['output']
        print(f"\n📄 設定ファイルの情報:")
        print(f"  - SP-API入力ファイル: {config['sp_api_input']}")
        print(f"  - Keepa入力ファイル: {config['keepa_input']}")
        print(f"  - 出力ファイル: {config['output_file']}")
        
        # 追加ソース情報を表示
        sources = integrator.config['data_integration'].get('sources', [])
        if sources:
            print("\n📄 追加ソース情報:")
            for i, source in enumerate(sources, 1):
                source_type = source.get('type', '不明')
                files = source.get('files', [])
                print(f"  ソース{i} ({source_type}): {', '.join(files)}")
        
        # SP-APIとKeepaデータの読み込み
        sp_df, keepa_df = integrator.load_data()
        print(f"📊 SP-APIデータ: {len(sp_df)}件")
        print(f"📊 Keepaデータ: {len(keepa_df)}件")
        
        # keepa_dfを保存（JAN-ASINマッピング用）
        integrator.keepa_df = keepa_df
        
        # SP-APIとKeepaデータの結合
        merged_df = integrator.merge_data(sp_df, keepa_df)
        
        # 追加ソースデータの結合
        for source_config in sources:
            source_type = source_config.get('type', '不明')
            print(f"\n📊 ソースデータ読み込み ({source_type})")
            
            # ソースデータの読み込み
            source_data = integrator.load_source_data(source_config)
            
            if source_data:
                # ソースデータの結合
                merged_df = integrator.merge_source_data(merged_df, source_data, source_config)
                print(f"✅ {source_type}データの結合完了: 現在 {len(merged_df.columns)}列")
        
        # カラムの並び替え
        merged_df = integrator.rearrange_columns(merged_df)
        
        # 統合データの保存
        integrator.save_data(merged_df)
        
        # カラム構成の確認
        print("\n=== 統合後のカラム構成 ===")
        print(f"\n全カラム数: {len(merged_df.columns)}")
        
        # 結果の統計情報
        print("\n=== 統合結果の概要 ===")
        print(f"・総データ件数: {len(merged_df)}件")
        print(f"・カラム数: {len(merged_df.columns)}列")
        
        # サンプルデータの表示
        print("\n=== 統合結果のサンプル（最初の3件）===")
        pd.set_option('display.max_columns', None)
        print(merged_df.head(3))
        
        print(f"\n✨ 処理完了！ データを {config['output_file']} に保存しました")
        
    except Exception as e:
        print(f"❌ エラーが発生しました: {str(e)}")
        logging.error(f"実行時エラー: {str(e)}")
        import traceback
        traceback.print_exc()

設定ファイルを読み込みました: C:\Users\inato\Documents\amazon-research\config\settings.yaml
📄 ログファイル: C:\Users\inato\Documents\amazon-research\logs\integration_20250407_141356.log
📂 プロジェクトルートディレクトリ: C:\Users\inato\Documents\amazon-research
📂 データディレクトリ: C:\Users\inato\Documents\amazon-research\data
📂 ログディレクトリ: C:\Users\inato\Documents\amazon-research\logs

📄 設定ファイルの情報:
  - SP-API入力ファイル: sp_api_output_filtered.csv
  - Keepa入力ファイル: keepa_output.csv
  - 出力ファイル: integrated_data.csv

📄 追加ソース情報:
  ソース1 (netsea): netsea_scraping.csv
  ソース2 (sudeli): sudeli_scraping.csv
  ソース3 (yahoo): yahoo_shopping_items.csv
  ソース4 (yoriyasu): yoriyasu_prices.csv
📊 SP-APIデータ: 99件
📊 Keepaデータ: 99件

📊 ソースデータ読み込み (netsea)
📊 netsea_scraping.csvを読み込みました: 316件, キー列: JANコード
📊 netseaデータから'JANコード'列を'JAN'として結合: netsea_scraping.csv
ℹ️ netsea_scraping.csv内に16件の重複JANを検出: 4980901211667, 4529052003808, 4970883013632など
ℹ️ 重複JANは各JANの最初のデータのみを使用します
ℹ️ 重複を除去: 316行 → 300行 (16行削除)
⚠️ マッチするJANコードがありません。結合をスキップします。
✅ netseaデータの結合完了: 現在 83列
✅ net